## Reference
JSON in Python: https://www.geeksforgeeks.org/json-with-python/ \
Main reference: https://ryankemmer.medium.com/algorithmic-trading-with-python-and-td-ameritrade-ec00fb47f57 \
https://github.com/alexgolec/tda-api \
https://developer.tdameritrade.com/home

Robinhood Tradebot: https://www.activestate.com/blog/how-to-build-an-algorithmic-trading-bot/ \
Interactive Broker Tradebot: https://algotrading101.com/learn/interactive-brokers-python-api-native-guide/ \

In [1]:
#Packages needed for current project

#pip install tda-api
#pip install webdriver-manager 
#pip install selenium
#pip install schedule

from tda import auth, client
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
from datetime import datetime
import schedule, time
import pandas as pd
import numpy as np
from statistics import mean
from statistics import stdev
from tda.orders.equities import equity_buy_market, equity_sell_market
from tda.orders.common import Duration, Session


### A. Authentication

In [4]:
def auth_func():

    token_path = 'token.pickle.shany'
    api_key = 'QXXA0K3LVUDBPX3CAJHKRCK99WCI5GTB'
    callback_url = 'https://localhost'
    try:
        c = auth.client_from_token_file(token_path, api_key)
    except FileNotFoundError:
        from selenium import webdriver
        with webdriver.Chrome(ChromeDriverManager().install()) as driver:
            c = auth.client_from_login_flow(
                driver,api_key, callback_url , token_path)
    '''
    r = c.get_price_history('AAPL',
        period_type=client.Client.PriceHistory.PeriodType.YEAR,
        period=client.Client.PriceHistory.Period.ONE_YEAR,
        frequency_type=client.Client.PriceHistory.FrequencyType.WEEKLY,
        frequency=client.Client.PriceHistory.Frequency.WEEKLY)
    assert r.status_code == 200, r.raise_for_status()
    x = json.dumps(r.json(), indent=4)     #Type of r.json() is dict. / json.dump(x,y) converts dict. to JSON data type
    y = r.json()
    z = y['candles']                       #Selecting column/data from dict. calls the category name
    zz = pd.DataFrame(z)
    zzz = zz.datetime
    print(type(zzz))
    print(zz)
    #####
    y = json.dumps(z)
    df = pd.read_json(y)                   #Converts datetime from microxxx to normal datetime
    print(df) '''
    return c


In [5]:
auth_func()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - There is no [win32] chromedriver for browser 103.0.5060 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\parka\.wdm\drivers\chromedriver\win32\103.0.5060.134]
<ipython-input-4-73a173c93ae0>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(ChromeDriverManager().install()) as driver:



**************************************************************

Opening the login page in a webdriver. Please use this window to log in. Successful login will be detected automatically.

If you encounter any issues, see here for troubleshooting: https://tda-api.readthedocs.io/en/stable/auth.html#troubleshooting

**************************************************************



RedirectTimeoutError: timed out waiting for redirect

### B. Trading Strategy/ Algorithm 

In [51]:
#function to calculate moving averages
#prices - pandas df of historical price
#time_period - int, specifies lookback period
def get_MovingAverage(prices, time_period):

    prices = prices[-time_period:]
    ma = mean(prices)
    return ma

#function to calculate bollinger bands
#prices - pandas df of historical price
#time_period - int, specifies lookback period
def get_BBands(prices, time_period):

    prices = prices[-time_period:]
    print(prices)
    ma = get_MovingAverage(prices, time_period)
    std = stdev(prices)
    bup = ma + 2*std
    bdown = ma - 2*std
    return bup, bdown

#function to retrieve previous 30 minute closing prices 
#c - user authentication token
#end - datetime of current time
def get_prices(c, end, STOCK):

    r = c.get_price_history(STOCK,
                            period_type=client.Client.PriceHistory.PeriodType.DAY,
                            period=client.Client.PriceHistory.Period.THREE_DAYS,
                            frequency_type=client.Client.PriceHistory.FrequencyType.MINUTE,
                            frequency=client.Client.PriceHistory.Frequency.EVERY_THIRTY_MINUTES,
                            end_datetime=end,
                            need_extended_hours_data=False
                            )

    assert r.status_code == 200, r.raise_for_status()

    y = r.json()
    y = y["candles"]
    y = json.dumps(y)
    df = pd.read_json(y)
    # drop last row
    df = df[:-1]

    return df

### B2. Other Info. 

In [52]:
def get_cur_price(c, STOCK):
        r = c.get_quote(STOCK)
        assert r.status_code == 200, r.raise_for_status()
        y = r.json()
        print(y)
        price = y[STOCK]["lastPrice"]
        print(price)
        return price


In [61]:
def get_account(c):
    r = c.get_account(ACCT_NUMBER)
    assert r.status_code == 200, r.raise_for_status()

    y = r.json()
    print(y)
    balance = y['securitiesAccount']['currentBalances']['cashBalance']
    roundtrips = y['securitiesAccount']['roundTrips']

    return balance, roundtrips

#see if we currently have a position 
def get_position(c):

    r = c.get_account(ACCT_NUMBER, fields=c.Account.Fields.POSITIONS)
    assert r.status_code == 200, r.raise_for_status()

    y = r.json()

    if "positions" in y["securitiesAccount"]:
        print(y["securitiesAccount"]["positions"][0])
        print(len(y["securitiesAccount"]["positions"]))
        print(y["securitiesAccount"]["positions"][0]["instrument"])
        return True
    else:
        return False


### B3. Placing Orders

In [53]:
def place_order(c, order_type, shares, STOCK):

    if order_type == 'buy':
        order_spec = equity_buy_market(STOCK, shares).set_session(
            Session.NORMAL).set_duration(Duration.DAY).build()
        c.place_order(ACCT_NUMBER, order_spec)

    if order_type == 'sell':
        order_spec = equity_sell_market(STOCK, shares).set_session(
            Session.NORMAL).set_duration(Duration.DAY).build()
        c.place_order(ACCT_NUMBER, order_spec)

### C. Execution

In [54]:
def get_action():

    c = auth_func()
    now = datetime.now()
    print(now)

    try:
        # get current position
        position = get_position(c)
        print('HAS POSITION: ' + str(position))
        
        for i in range(len(Look)):
            hi = Look[i]
            print(hi)
            df = get_prices(c, now, hi)
            bup, bdown = get_BBands(df.close, TIME_PERIOD)

            # get current price
            price = get_cur_price(c, hi)

            # get account balance
            balance, roundtrips = get_account(c)
            
            print(str(i) + "th stock")
            print("Current balance " + str(balance))
            print("Current price " + str(price))
            print("High Band " + str(bup))
            print("Low Band " + str(bdown))

            # check if roundtrips is less than 2
            '''if roundtrips < 2:

                if price < bdown:
                    if position == False:
                        place_order(c, 'buy', 1, hi)
                        print("Bought")

                if price > bup:
                    if position == True:
                        place_order(c, 'sell', 1, hi)
                        print("Sold")
            '''
    except:
        print('Auth error')

In [66]:
def main():

    schedule.every().day.at("07:00").do(get_action)
    schedule.every().day.at("07:30").do(get_action)
    schedule.every().day.at("08:00").do(get_action)
    schedule.every().day.at("08:30").do(get_action)
    schedule.every().day.at("09:00").do(get_action)
    schedule.every().day.at("09:30").do(get_action)
    schedule.every().day.at("10:00").do(get_action)
    schedule.every().day.at("10:30").do(get_action)
    schedule.every().day.at("11:54").do(get_action)
    schedule.every().day.at("11:58").do(get_action)
    schedule.every().day.at("12:08").do(get_action)
    schedule.every().day.at("12:30").do(get_action)
    schedule.every().day.at("02:07").do(get_action)

    while True:
        schedule.run_pending()
        time.sleep(1)

### C1. Deploy Bot

In [67]:
Look = ['AAPL','META']
TIME_PERIOD = 10
ACCT_NUMBER = '275802489'


In [68]:
main()

2022-06-17 02:07:00.900249
{'shortQuantity': 0.0, 'averagePrice': 244.57291, 'currentDayCost': 0.0, 'currentDayProfitLoss': -743.599999999999, 'currentDayProfitLossPercentage': -2.69, 'longQuantity': 110.0, 'settledLongQuantity': 110.0, 'settledShortQuantity': 0.0, 'instrument': {'assetType': 'EQUITY', 'cusip': '594918104', 'symbol': 'MSFT'}, 'marketValue': 26950.0, 'maintenanceRequirement': 8085.0, 'previousSessionLongQuantity': 110.0}
12
{'assetType': 'EQUITY', 'cusip': '594918104', 'symbol': 'MSFT'}
HAS POSITION: True
AAPL
28    130.1281
29    130.9100
30    130.8900
31    131.0100
32    131.0400
33    131.2550
34    130.0200
35    129.6850
36    129.5500
37    130.1150
Name: close, dtype: float64
{'AAPL': {'assetType': 'EQUITY', 'assetMainType': 'EQUITY', 'cusip': '037833100', 'assetSubType': '', 'symbol': 'AAPL', 'description': 'Apple Inc. - Common Stock', 'bidPrice': 130.53, 'bidSize': 100, 'bidId': 'P', 'askPrice': 130.55, 'askSize': 300, 'askId': 'P', 'lastPrice': 130.53, 'last

KeyboardInterrupt: 

## Trial

In [74]:
from sklearn.linear_model import LinearRegression

xVals = np.linspace(4,14,101)
print(xVals)
print(len(xVals))
x1 = np.array([10,8,13,9,11,14,6,4,12,7,5])
y1 = np.array([8.04,6.95,7.58,8.81,8.33,9.96,7.24,4.26,10.84,4.82,5.68])
model = LinearRegression().fit(x1.reshape(-1,1),y1.reshape(-1,1))
print(model)
print(model.coef_)

[ 4.   4.1  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3
  5.4  5.5  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7
  6.8  6.9  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1
  8.2  8.3  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5
  9.6  9.7  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9
 11.  11.1 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.  12.1 12.2 12.3
 12.4 12.5 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7
 13.8 13.9 14. ]
101
LinearRegression()
[[0.50009091]]
